# 目的
kerasでのssplit実験に向けた動作テスト用作業ノート。


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Reshape,Convolution1D
from keras.utils import np_utils
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [52]:
model = Sequential()
model.add(Embedding(10000, 4, input_length=5))
model.add(Convolution1D(2, 3, border_mode='same', input_shape=(4, 5)))
model.add(Convolution1D(3, 1, border_mode='same', input_shape=(2, 5)))
model.add(Activation('relu'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [53]:
X_train = np.random.randint(10,size=(1,5))
y_train = np.random.randint(3,size=(1,5))

In [54]:
X_train = X_train.reshape(1,5)

In [55]:
Y_train = y_train.reshape(1,5)

In [56]:
X_train

array([[8, 1, 2, 3, 6]])

In [57]:
Z_train = model.predict(X_train)

In [58]:
Z_train.shape

(1, 5, 3)

In [59]:
Z_train

array([[[ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.00301621],
        [ 0.00376692,  0.0062562 ,  0.00182627],
        [ 0.00884872,  0.02521124,  0.        ],
        [ 0.00534717,  0.0093908 ,  0.00150314]]], dtype=float32)

In [67]:
Y_train = np_utils.to_categorical(Y_train[0],3)

In [70]:
Y_train = Y_train.reshape(1,5,3)

In [71]:
model.fit(X_train,Y_train,verbose=1)

Epoch 1/10
1/1 [==============================] - 0s - loss: 7.0510 - acc: 0.4000
Epoch 2/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.2000
Epoch 3/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.2000
Epoch 4/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.2000
Epoch 5/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.2000
Epoch 6/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.2000
Epoch 7/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.2000
Epoch 8/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.2000
Epoch 9/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.2000
Epoch 10/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 0.2000


In [73]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_6 (Embedding)          (None, 5, 4)          40000       embedding_input_6[0][0]          
____________________________________________________________________________________________________
convolution1d_6 (Convolution1D)  (None, 5, 2)          26          embedding_6[0][0]                
____________________________________________________________________________________________________
convolution1d_7 (Convolution1D)  (None, 5, 3)          9           convolution1d_6[0][0]            
____________________________________________________________________________________________________
activation_6 (Activation)        (None, 5, 3)          0           convolution1d_7[0][0]            
Total params: 40035
_______________________________________________________________________

各文字を畳み込みを用いて３クラスに分類し、crossentropyを損失関数として学習を行う準備は以上。
あとは、計算過程の確認をして、想定通りに動くことを順次確認していく。

## Embedding
Embeddingは入力の文字ID列$D=d_0,d_1,\cdots,d_M$に対して、対応する重み付きベクトル$\vec{v_i} =NET_{Emb}(d_i)$が取得でき、その重みが更新されることが要求として定義できる。
一層のmodelを作成して、任意の整数値を入力し、その順伝搬計算結果と重みベクトル配列の関係を確認する。

In [86]:
modelEmbedding = Sequential()
modelEmbedding.add(Embedding(10000, 4, input_length= 5))
modelEmbedding.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [87]:
X_input = np.array((1,2,3,4,5))
X_input = X_input.reshape(1,5)

In [88]:
Y_output = modelEmbedding.predict(X_input)

In [89]:
Y_output

array([[[ 0.04938814, -0.02848035,  0.01871261,  0.04999331],
        [-0.02083539,  0.03934289,  0.04745951, -0.03342854],
        [ 0.02362558,  0.00391078, -0.0305439 ,  0.02737233],
        [-0.02744209, -0.04912445, -0.02523756,  0.04928442],
        [-0.03242682, -0.02965616,  0.04361621, -0.03901239]]], dtype=float32)

In [92]:
embWeight = modelEmbedding.get_weights()

In [95]:
print(embWeight[0][1])
print(embWeight[0][2])
print(embWeight[0][3])
print(embWeight[0][4])
print(embWeight[0][5])

[ 0.04938814 -0.02848035  0.01871261  0.04999331]
[-0.02083539  0.03934289  0.04745951 -0.03342854]
[ 0.02362558  0.00391078 -0.0305439   0.02737233]
[-0.02744209 -0.04912445 -0.02523756  0.04928442]
[-0.03242682 -0.02965616  0.04361621 -0.03901239]


一致していることが確認できた。

## Convolution 1D（中間層）
$N\times M$のデータを入力し、指定した出力チャネル数$\text{Ch}_{output}$に応じた$\text{Ch}_{output} \times M$の出力が得られることが要求。また、その際に対象文字$v_i$を中心とした場合の前後$W$文字数の畳み込みとなっていることを確認する。

In [177]:
modelConvInter = Sequential()
modelConvInter.add(Convolution1D(1,3, border_mode='same',input_shape=(5,4)))
modelConvInter.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [178]:
Y_output.shape

(1, 5, 4)

In [179]:
Z_output = modelConvInter.predict(Y_output)

In [180]:
Z_output

array([[[-0.04335489],
        [ 0.01496695],
        [-0.05528253],
        [-0.0237015 ],
        [-0.02779479]]], dtype=float32)

In [181]:
convInterWeight = modelConvInter.get_weights()

In [182]:
convInterWeight = convInterWeight[0]

In [183]:
convInterWeight

array([[[[ 0.43841523],
         [ 0.16669995],
         [-0.46943867],
         [-0.16445476]]],


       [[[-0.14824647],
         [ 0.42196745],
         [-0.56606954],
         [-0.54032665]]],


       [[[ 0.24413842],
         [ 0.41017324],
         [-0.1336731 ],
         [-0.2657384 ]]]], dtype=float32)

対象文字（中央）に対して作用される畳み込みベクトルは

In [203]:
convInterWeight[1]

array([[[-0.14824647],
        [ 0.42196745],
        [-0.56606954],
        [-0.54032665]]], dtype=float32)

となる。borderの設定を```same```としているため、領域外はゼロパディングが行われれている。

したがって、下記の入力データにおいて、

In [204]:
Y_output

array([[[ 0.04938814, -0.02848035,  0.01871261,  0.04999331],
        [-0.02083539,  0.03934289,  0.04745951, -0.03342854],
        [ 0.02362558,  0.00391078, -0.0305439 ,  0.02737233],
        [-0.02744209, -0.04912445, -0.02523756,  0.04928442],
        [-0.03242682, -0.02965616,  0.04361621, -0.03901239]]], dtype=float32)

１番目の文字に対しては、畳み込みフィルタの２列目と１番目が、３列目と２番目の要素ごとの積の総和と一致するはずである。

In [205]:
a = np.sum(np.multiply(convInterWeight[1].reshape(4), Y_output[0][0]))

In [206]:
b = np.sum(np.multiply(convInterWeight[2].reshape(4), Y_output[0][1]))

In [207]:
a + b == Z_output[0][0][0]

True

一致した。これでフィルタ数１個の場合は理解できた。

次はフィルタ数が２個以上の場合、要素の順番と計算順序の理解を行う。

In [208]:
modelConvInter = Sequential()
modelConvInter.add(Convolution1D(2,3, border_mode='same',input_shape=(5,4)))
modelConvInter.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [209]:
Z_output = modelConvInter.predict(Y_output)

In [210]:
Z_output

array([[[ 0.01080527, -0.01493756],
        [ 0.04832333,  0.0315503 ],
        [-0.00605015,  0.01602747],
        [-0.05350912,  0.01529768],
        [-0.00605088, -0.00430768]]], dtype=float32)

In [215]:
convInterWeight = modelConvInter.get_weights()[0]

In [226]:
convInterWeight[0][0]

array([[  4.06886786e-01,   3.92321855e-01],
       [  3.59058380e-04,   3.69453460e-01],
       [  4.14357334e-01,  -3.89175117e-02],
       [ -1.85132757e-01,   2.77574986e-01]], dtype=float32)

In [227]:
convInterWeight[0][0][1][0]

0.00035905838

In [238]:
convInterWeight

array([[[[  4.06886786e-01,   3.92321855e-01],
         [  3.59058380e-04,   3.69453460e-01],
         [  4.14357334e-01,  -3.89175117e-02],
         [ -1.85132757e-01,   2.77574986e-01]]],


       [[[  1.96072370e-01,  -9.02287066e-02],
         [  6.26488030e-02,  -2.48909131e-01],
         [  3.72217327e-01,   1.12524480e-01],
         [ -4.27949160e-01,   1.35087162e-01]]],


       [[[ -3.78177166e-02,  -3.63870561e-02],
         [  3.03483039e-01,  -4.41676974e-02],
         [  3.44903797e-01,  -3.81215990e-01],
         [  3.51839870e-01,   2.20098406e-01]]]], dtype=float32)

１番目のフィルタと１番目の文字の畳み込みは 

In [256]:
s = 0
for i in range(0,4):
    s += convInterWeight[1][0][i][0] * Y_output[0][0][i]
    s += convInterWeight[2][0][i][0] * Y_output[0][1][i]

となる。この計算結果と

In [257]:
Z_output[0][0][0]

0.010805272

が一致することが条件。
計算結果は

In [258]:
s

0.010805272904690355

であるため、一致することが確認できた。（精度の部分で一致していないが）

２番目のフィルタとは、

In [259]:
s2 = 0
for i in range(0,4):
    s2 += convInterWeight[1][0][i][1] * Y_output[0][0][i]
    s2 += convInterWeight[2][0][i][1] * Y_output[0][1][i]

In [260]:
Z_output[0][0][1]

-0.014937565

In [261]:
s2

-0.014937565254513174

一致している。これによりConvolution1Dの計算過程について確認できた。

## Flatten
Flatten処理後のオーダーを確認する。

In [4]:
X_input = np.array([[1,2,3],[4,5,6]])
X_input = X_input.reshape(1,2,3)
X_input

array([[[1, 2, 3],
        [4, 5, 6]]])

In [5]:
modelFlatten = Sequential()
modelFlatten.add(Flatten(input_shape=(2,3)))
modelFlatten.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [6]:
Y_output = modelFlatten.predict(X_input)
Y_output

array([[ 1.,  2.,  3.,  4.,  5.,  6.]], dtype=float32)

行の方向に沿って連結される。

## 計算手順
1. 入力文字列長をMに固定。不足分はゼロパディング。
1. Embeddingで12chのベクトルにする。
1. Convolution1Dで128ch, W幅の畳み込みを行う。出力は128 x M。
1. Convolution1Dで3ch、幅＝１の畳み込みを行う。この計算はJuliaのLinearと同値。出力は3 x M
1. 活性化。
1. 誤差計算

上記手順に従ったモデルを作成。文字列長は10,000としておく。

In [312]:
modelSsplit = Sequential()
modelSsplit.add(Embedding(10000, 12, input_length=10000))
modelSsplit.add(Convolution1D(128, 5, border_mode='same', input_shape=(10000, 12)))
modelSsplit.add(Activation('relu'))
modelSsplit.add(Convolution1D(3, 1, border_mode='same', input_shape = (10000, 128)))
modelSsplit.add(Activation('softmax'))
modelSsplit.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [313]:
X_input = np.array(range(0,10000))
X_input = X_input.reshape(1,10000)

In [315]:
modelSsplit.predict(X_input)

array([[[ 0.33274397,  0.3334181 ,  0.33383793],
        [ 0.33339199,  0.33323509,  0.33337295],
        [ 0.33305755,  0.33347532,  0.3334671 ],
        ..., 
        [ 0.3332012 ,  0.33334255,  0.33345628],
        [ 0.33326969,  0.33356619,  0.33316413],
        [ 0.33316839,  0.3337341 ,  0.33309755]]], dtype=float32)

In [322]:
X_train = np.random.randint(10,size=(10,10000))
y_train = np.random.randint(3,size=(10,10000))

In [323]:
Y_train = np_utils.to_categorical(y_train[0],3)

In [324]:
y_train.reshape(10*10000)

array([1, 2, 1, ..., 0, 1, 2])

In [325]:
Y_train = np_utils.to_categorical(y_train.reshape(10*10000),3).reshape(10,10000,3)

In [326]:
modelSsplit.fit(X_train,Y_train,nb_epoch=10, verbose=1)

Epoch 1/10
10/10 [==============================] - 0s - loss: 1.0986 - acc: 0.3315
Epoch 2/10
10/10 [==============================] - 0s - loss: 1.0986 - acc: 0.3312
Epoch 3/10
10/10 [==============================] - 0s - loss: 1.0986 - acc: 0.3320
Epoch 4/10
10/10 [==============================] - 0s - loss: 1.0986 - acc: 0.3337
Epoch 5/10
10/10 [==============================] - 0s - loss: 1.0986 - acc: 0.3345
Epoch 6/10
10/10 [==============================] - 0s - loss: 1.0986 - acc: 0.3350
Epoch 7/10
10/10 [==============================] - 0s - loss: 1.0986 - acc: 0.3363
Epoch 8/10
10/10 [==============================] - 0s - loss: 1.0986 - acc: 0.3378
Epoch 9/10
10/10 [==============================] - 0s - loss: 1.0986 - acc: 0.3382
Epoch 10/10
10/10 [==============================] - 0s - loss: 1.0986 - acc: 0.3381


In [327]:
modelSsplit.predict(X_input)

array([[[ 0.33209026,  0.33381799,  0.33409178],
        [ 0.33263537,  0.33358216,  0.33378246],
        [ 0.33215046,  0.33395809,  0.33389145],
        ..., 
        [ 0.33236489,  0.33389416,  0.33374092],
        [ 0.33237731,  0.33395407,  0.33366862],
        [ 0.33256733,  0.33403474,  0.33339787]]], dtype=float32)

# 実験準備
ネットワークの構築方法、学習の設定などについては確認できた。次は、実際の実験に向けた準備を行う。

要求は下記のとおり。
- 文字のルックアップテーブル用データ
- コーパスデータ